# 2. Data reading and Cleaning

In [ ]:
#import librarys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
import dataframe_image as dfi
from functools import reduce #
from datetime import datetime, timedelta
from scipy.stats import chi2_contingency
from functools import reduce #import reduce from functools


In [ ]:
#read data
airports = pd.read_csv("airports.csv") #airport variables
carriers = pd.read_csv("carriers.csv") #carrier variables
airplanes = pd.read_csv("plane-data.csv") #airplane variables

df1 = pd.read_csv("2006.csv") #import data for year 2006
df2 = pd.read_csv("2007.csv") #import data for year 2007
df3 = pd.read_csv("2008.csv") #import data for year 2008

In [ ]:
#see and compare data shape
print(df1.shape) #year 2006
print(df2.shape) #year 2007
print(df3.shape) #year 2008

In [ ]:
#check for missing datapoints in 2008.csv (due to the small size compared to other years)
df3['Month'].value_counts() 

In [ ]:
#year 2008 only has month up to april, so we don't use it.

#check for NA's for 2006 and 2007
print(df1.isna().sum())
print(df2.isna().sum())

If we adress NA's now, i it could potentially lead to loss of important datapoints. Let's instead try to see if there is a rational explanation for some of theese, i.e. if a flight is cancelled, then any delay entry shouldn't be possible. 

In [1]:
#check for correlation between cancellation and delays 

#merge dataframes for 2006 and 2007
cancel_org = pd.concat([df1, df2],ignore_index=True) #we can also use to verify NA's later.

#select columns of interest
cancel=cancel_org[['Cancelled','CancellationCode','ArrDelay','DepDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']]

#correlation heatmap (check how cancelations are correlated to delays)
cancel_heatm = sns.heatmap(cancel.corr()) #plot the heatmap
cancel_heatm
#cancel_heatm.figure.savefig("clean_plot1.png") #save to img

NameError: name 'pd' is not defined

In [ ]:
#the heatmap suggest a very strong correlation

#narrow in on the issue
cancel2 = cancel[['Cancelled','CancellationCode','ArrDelay','DepDelay']]
cancel2=cancel2[cancel2['Cancelled']==1] #  select only cancelled flights

#explore cancelation data
print(cancel2)
print(cancel2.isna().sum()) #sum of NA's

In [ ]:
#dataframes without cancelled flights.
df1=df1[df1['Cancelled']==0] 
df2=df2[df2['Cancelled']==0] 

#Similarly if a plane is diverted, then arrival delay entry should be impossible.
#remaining NA values corresponds to diverted in the same way as cancelled.
print(df1.ArrDelay.isna().sum()) #sum of NA's
print(df1.DepDelay.isna().sum()) #sum of NA's
print(df2.ArrDelay.isna().sum()) #sum of NA's
print(df2.DepDelay.isna().sum()) #sum of NA's

In [ ]:
#heatmap of correlation between diverted and delays

#columns of interest
diverted=cancel_org[['Diverted','CancellationCode','ArrDelay','DepDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']]

#correlation heatmap
diverted_heatm = sns.heatmap(diverted.corr()) #plot the heatmap
diverted_heatm
#diverted_heatm.figure.savefig("clean_plot2.png") #save to img

In [ ]:
#again we see a very strong correlation. so we select non-diverted flights.

#dataframes without diverted flights.
df1=df1[df1['Diverted']==0] 
df2=df2[df2['Diverted']==0] 

#if our assumption regarding diverted is true, NA's should excluded from ArrDelay and DepDelay
print(df1.ArrDelay.isna().sum()) #sum of NA's
print(df1.DepDelay.isna().sum()) #sum of NA's
print(df2.DepDelay.isna().sum()) #sum of NA's
print(df2.ArrDelay.isna().sum()) #sum of NA's

In [ ]:
#see shape of data
print(df1.shape)
print(df2.shape)

In [ ]:
#to avoid any potential bias towards one year, we take sample to match datapoints
df2 = df2.sample(7003802)

In [ ]:
#merge data (exclude 2008 to avoid bias towards the first 4 months)
df = pd.concat([df1, df2],ignore_index=True)

#verify that missing values in cancelled is taken care of
print(df['Cancelled'].sum())

#test against original data
print(cancel_org['Cancelled'].sum())

In [ ]:
#explore the data
df 

In [ ]:
#merge df with carrier variables to get names of airlines
df = pd.merge(df, carriers,  how='inner', left_on=['UniqueCarrier'], right_on = ['Code'])

#rename Description column to Carrier
df.rename(columns={'Description': 'Carrier'}, inplace=True)

#create columns
df['delay'] = ((df['DepDelay'])+(df['ArrDelay'])) #total delay column for a flight (Departure + Arrival delay)
df['dep_hour_standard'] = df['DepTime']/100*1.0386100 #departure standard time column
df['Date']= pd.to_datetime(pd.DataFrame({'year':df['Year'],
                                         'month':df['Month'],'day':df['DayofMonth'], 
                                         'hour':df['dep_hour_standard']})) #date column

df['dep_hour_int'] = (df['DepTime']/100).astype(int) #departure time column (in integer)
df['Date_bin']= pd.to_datetime(pd.DataFrame({'year':df['Year'],
                                             'month':df['Month'],'day':df['DayofMonth'], 
                                             'hour':df['dep_hour_int']})) #date column in bins

df.head() #explore new columns

# 3. EDA

In [ ]:
#details of delay related columns
del_info = df[['ArrDelay','DepDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay',
               'LateAircraftDelay','delay']] #select columns
del_info.rename(columns={'ArrDelay': 'Arrival delay', 'DepDelay': 'Departure Delay',
                         'CarrierDelay':'Carrier Delay','WeatherDelay':'Weather Delay',
                         'NASDelay': 'NAS Delay', 'SecurityDelay': 'Security Delay',
                         'LateAircraftDelay':'Late Aircraft Delay',
                         'delay':'Total Delay'}, inplace=True) #rename columns
del_info = del_info.describe().T #describe and transpose delay information

#export to image 
del_info_style = del_info.style.format("{:.12}").background_gradient() #style table
dfi.export(del_info_style,"EDA_table1.png") #export to img
del_info_style

In [ ]:
#explore departure delays (i.e. outlier, etc.)
print(df["DepDelay"][(df["DepDelay"] <0)].count())
print(df["DepDelay"][(df["DepDelay"] < -1000)].count())
print(df["DepDelay"][(df["DepDelay"] >2000)].count())

#explore arrival delays
print(df["ArrDelay"][(df["ArrDelay"] <0)].count())
print(df["ArrDelay"][(df["ArrDelay"] < -400)].count())
print(df["ArrDelay"][(df["ArrDelay"] >2000)].count())

In [ ]:
#explore departure and arrival max outliers
exp_del = df[['Carrier','TailNum','DepDelay','ArrDelay']]
exp_del = exp_del[(exp_del["ArrDelay"] >2000) & (exp_del["DepDelay"] >2000)]

#export outlier table
exp_del_style = exp_del.style.format("{:.12}").background_gradient() #style table
dfi.export(exp_del_style,"EDA_table1_2.png") #export to img
exp_del_style 

#### Percentage of all delays


In [ ]:
#total no. of flights ontime, and delayed
delay_perc = (df['delay']<=0) #true: before or ontime, false: delayed
delay_perc.value_counts() #count

In [ ]:
#manual math cell: percentage of delayed flights
tot_flights = (6534546+7473058)
del_flights = 6534546
ont_flights = 7473058
perc=del_flights/tot_flights
perc2=ont_flights/tot_flights
print("Percentage of flights that are delayed: {:.00%}".format(perc))
print("Percentage of flights that are ontime: {:.00%}".format(perc2))

In [ ]:
#change the total delay threshold to 10 minutes:
delay_perc2 = (df['delay']<=10) #10 min delay or less.
delay_perc2.value_counts() #count

In [ ]:
#manual math cell: percentage of delayed flights (10min  threshold)
tot_flights = (4671113+9336491)
del_flights = 4671113
ont_flights = 9336491
perc=del_flights/tot_flights
perc2=ont_flights/tot_flights
print("Percentage of flights that are delayed: {:.00%} - (10min threshold)".format(perc))
print("Percentage of flights that are ontime: {:.00%} - (10min threshold)".format(perc2))

In [ ]:
#set plotting style for notebook
sns.reset_orig()
plt.style.use('seaborn-colorblind')
sns.set_style("whitegrid")

In [ ]:
#plot delays

#data prep for plot
#percentage of delays converted to true/false values
delay_perc_plot = pd.DataFrame() #empty df
delay_perc_plot['Status']=df['delay']<=0 #column with delay status
for col in delay_perc_plot.columns[delay_perc_plot.dtypes == 'bool']:
    delay_perc_plot['Status'] = delay_perc_plot['Status'].map({True: 'On time', False: 'Delayed'}) #convert to boolean

#percentage of delays with a 10 min threshold (true/false)
delay_perc_plot_10 = pd.DataFrame() #cempty df
delay_perc_plot_10['Status_10min_int']=df['delay']<10 #10min threshold
for col in delay_perc_plot_10.columns[delay_perc_plot_10.dtypes == 'bool']:
    delay_perc_plot_10['Status_10min_int'] = delay_perc_plot_10['Status_10min_int'].map({True: 'On time', False: 'Delayed'}) #convert to boolean


#combined plot
f, ax = plt.subplots(1,2,figsize=(15, 7))

palette=['C2','C0'] #set colour

(delay_perc_plot['Status'].value_counts(normalize=True)*100).sort_values().plot(kind = 'bar', ax=ax[0], color=palette) #plot in percentage
(delay_perc_plot_10['Status_10min_int'].value_counts(normalize=True)*100).sort_values().plot(kind = 'bar', ax=ax[1], color=palette) #plot in percentage
#set labels
ax[0].set_title("Percentage of flights: delay vs. on time",fontsize = 16) #title ax0
plt.setp(ax[0].get_xticklabels(), rotation=45, ha='right',fontsize = 12) #rotate ax0
ax[0].set(ylabel="Percentage")
ax[1].set_title("Percentage of flights: delay vs. on time (10min theshold)",fontsize = 16) #title ax1
plt.setp(ax[1].get_xticklabels(), rotation=45, ha='right',fontsize = 12) #rotate ax1
ax[1].set(ylabel="Percentage")
plt.tight_layout() #tight layout
plt.style.use('seaborn-colorblind')
plt.show() #plot

In [ ]:
#plot delays (total)

#data prep for plot
#total delay converted to true/false values
delay_tot_plot = pd.DataFrame() #empty df
delay_tot_plot['Status']=df['delay']<=0 #0 min threshold
for col in delay_tot_plot.columns[delay_tot_plot.dtypes == 'bool']:
    delay_tot_plot['Status'] = delay_tot_plot['Status'].map({True: 'On time', False: 'Delayed'}) #convert to boolean

#total delay 10min threshold
delay_tot_plot_10 = pd.DataFrame() #empty df
delay_tot_plot_10['Status_10min_int']=df['delay']<10 #10 min threshold
for col in delay_tot_plot_10.columns[delay_tot_plot_10.dtypes == 'bool']:
    delay_tot_plot_10['Status_10min_int'] = delay_tot_plot_10['Status_10min_int'].map({True: 'On time', False: 'Delayed'})

#plot of total delays
f, ax = plt.subplots(1,2,figsize=(15, 7))

palette=['C2','C0'] #color

(delay_tot_plot['Status'].value_counts()).sort_values().plot(kind = 'bar', ax=ax[0], color=palette)
(delay_tot_plot_10['Status_10min_int'].value_counts()).sort_values().plot(kind = 'bar', ax=ax[1], color=palette)

#set labels
ax[0].set_title("Total no. of flights: delay vs. ontime",fontsize = 16) #title ax0
plt.setp(ax[0].get_xticklabels(), rotation=45, ha='right',fontsize = 12) #rotate ax0
ax[1].set_title("Total no. of flights: vs. ontime (10min theshold)",fontsize = 16) #title ax1
plt.setp(ax[1].get_xticklabels(), rotation=45, ha='right',fontsize = 12) #rotate ax1
plt.setp(ax, ylabel="Total no. in 00000's") #common y label.
plt.tight_layout() #tight layout
plt.show() #plot

In [ ]:
#value count of both delay df's
print(delay_tot_plot.value_counts())
print(delay_tot_plot_10.value_counts())

In [ ]:
#merge and prettify the plots above

#plot data preparation
delay_dec_data=[7.473081,6.534523] #totals divided with 1000: fit plot to graph
delay_dec = pd.Series(index = ["On time", "Delayed"], data=delay_dec_data) #create series
percentage = delay_dec.div(delay_dec.sum()).mul(100).round(2) #calc. the percentage

#barplot with total and percentage in a combined plot (0 min threshold)
ax = delay_dec.plot(kind="bar", alpha=0.7, figsize=(10, 8), color=palette)

#set labels inside the bars
labels = [f'{delay_dec} - {percentage[delay_dec]}%' for delay_dec in delay_dec.index]
for label, a in zip(labels, ax.patches):
    left, bottom, width, height = a.get_bbox().bounds
    ax.annotate(label, xy=(left+width/2, bottom+height/2), ha='center', va='center', rotation=90, fontsize=28)
#labels
plt.ylabel("Count (mio.)", fontsize=25) #ylabel
plt.xlabel("Status", fontsize=25) #xlabel
plt.title("Delay vs. On time", fontsize=30, fontname="Monospace", alpha=.8) #title
plt.xticks([]) #empty xticks
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout (rectangular)
plt.savefig('EDA_plot1.png', bbox_inches='tight') #save img
plt.show()

#barplot with total and percentange in combined plot (10 min threshold)
delay_dec_data2=[9.190188,4.817416] #totals divided with 1000: to fit plot to graph
delay_dec2 = pd.Series(index = ["On time", "Delayed"], data=delay_dec_data2) #series
percentage2 = delay_dec2.div(delay_dec.sum()).mul(100).round(2) #calculate percent

#plot pretty bar for 10min threshold
ax = delay_dec2.plot(kind="bar",  alpha=0.7, figsize=(10, 8), color=palette)

#bar labels
labels = [f'{delay_dec2} - {percentage2[delay_dec2]}%' for delay_dec2 in delay_dec2.index]
for label, a in zip(labels, ax.patches):
    left, bottom, width, height = a.get_bbox().bounds
    ax.annotate(label, xy=(left+width/2, bottom+height/2), ha='center', va='center', rotation=90, fontsize=28)
#labels
plt.ylabel("Count (mio.)", fontsize=25) #ylabel
plt.xlabel("Status: Accept up to 10 min. delay", fontsize=25) #xlabel
plt.title("Delay vs. On time (<10 min.)", fontsize=30, fontname="Monospace", alpha=.8) #title
plt.xticks([]) #empty xticks
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout
plt.savefig('EDA_plot2.png', bbox_inches='tight') #save img
plt.show()

#### On time vs. delayed for each carrier 

In [ ]:
#plot of delay for each carrier (stacked barchart)

#data prep for plot
carier_delay_perc_plot = pd.DataFrame() #empty df
carier_delay_perc_plot['Carrier']=df['Carrier'] #carrier variable with full names
carier_delay_perc_plot['ontime']=df['delay']<=0 #on time vs. delay
carier_delay_perc_plot = carier_delay_perc_plot.groupby(['Carrier',
                                                           'ontime']).size().unstack(1) #shape and class data
carier_delay_perc_plot = carier_delay_perc_plot.div(carier_delay_perc_plot.sum(axis=1), axis=0) #percentage

#plot
f, ax = plt.subplots(figsize=(10, 8))
carier_delay_perc_plot.sort_values(by=False).plot(kind="bar", 
                                                  alpha=.9, stacked=True, ax=ax,color = palette) #stacked bar plot
#set labels
ax.set_title("Percentage of delay by carrier",fontsize = 30, fontname="Monospace", alpha=.8) #title
plt.xlabel('Carrier', fontsize = 16) #xlabel
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right") #rotate xlabel
plt.ylabel("Percentage", fontsize = 16) #ylabel
ax.legend(loc='upper right', title='On time', fontsize = 13)
plt.gca().set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()]) #yticks in percentage
plt.tight_layout(rect=[0, 0, 1, 1]) #tight rectangular layout
plt.savefig('EDA_plot3.png', bbox_inches='tight') #save img
plt.show() #plot

The percentage of delay by carrier could perhaps be somewhat misleading (i.e. Hawaiian Airlines most on time flights, but surely a carrier like Delta or SouthWest Airlines must have more flights!) Let's explore the data a little further:


In [ ]:
#create carrier total flights df
delay_carrier=df[['Carrier']].groupby("Carrier").count() #carrier column and group by count
delay_carrier['fligts']=df['Carrier'].value_counts() #total flights
delay_carrier = delay_carrier.reset_index(drop=False) #reset index

#carrier total delays df
delay_carrier2=(df[df['delay']>0]['Carrier'].value_counts()) #delayed
delay_carrier2=delay_carrier2.to_frame(name=('total delays')) #to df
delay_carrier2.index.name='Carrier' #set index name
delay_carrier2=delay_carrier2.reset_index(drop=False) #reset index

#merge to carrier delay df
delay_merge = delay_carrier.merge(delay_carrier2[['Carrier','total delays']],
                                  how='left',left_on='Carrier',right_on='Carrier')
delay_merge['percentage']=(delay_merge['total delays']/delay_merge['fligts']) #% delay column
delay_merge #explore new df

In [ ]:
#multiple plots to gain better insight in delay by carrier.

#plot total flights by carrier (should show that Hawaiian has way less flights than big carriers)
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data= delay_merge.sort_values(by='fligts'),ax=ax, x=('Carrier'), y=('fligts'))
ax.set_title('Total flights by carrier',fontsize = 30, fontname="Monospace", alpha=.8)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right") #rotate xlabel
ax.set_xlabel('Carrier',fontsize = 16)
ax.set_ylabel("Count (mio.)",fontsize = 16)
plt.tight_layout(rect=[0, 0, 1, 1])
plt.savefig('EDA_plot4.png', bbox_inches='tight') #save img
plt.show() #plot

#plot total delays by carrier
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data= delay_merge.sort_values(by='total delays'),ax=ax, x=('Carrier'), y=('total delays'))
ax.set_title('Total delays by carrier')
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right") #rotate xlabel
ax.set_xlabel('Carrier')
ax.set_ylabel('No. of total delays')
plt.tight_layout() #tight layout
plt.show() #plot

#plot percentage delays by carrier
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data= delay_merge.sort_values(by='percentage'),ax=ax, x=('Carrier'), y=('percentage'))
ax.set_title('Percentge of delays by carrier', fontsize=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right") #rotate xlabel
ax.set_xlabel('Carrier')
ax.set_ylabel('Percentage of delays')
plt.tight_layout() #tight layout
plt.show() #plot

In [ ]:
#merge df with airports, origin 
delay_by_airport = pd.merge(df, airports,  how='inner', left_on=['Origin'], right_on = ['iata'])
delay_by_airport #explore


In [ ]:
#create airport total flights df
delay_by_airport1 = delay_by_airport[['airport','iata']].groupby('airport').count()
delay_by_airport1.rename(columns={'iata': 'flights'}, inplace=True)
delay_by_airport1 = delay_by_airport1.sort_values(by=['flights'], ascending=False)
delay_by_airport1 = delay_by_airport1.reset_index(drop=False)

#carrier total delays df
delay_by_airport2=(delay_by_airport[delay_by_airport['delay']>0]['airport'].value_counts()) #delayed
delay_by_airport2=delay_by_airport2.to_frame(name=('total delays')) #to df
delay_by_airport2.index.name='airport' #set index name
delay_by_airport2=delay_by_airport2.reset_index(drop=False) #reset index

#merge to carrier delay df
delay_by_airport_merge = delay_by_airport1.merge(delay_by_airport2[['airport','total delays']],
                                  how='left',left_on='airport',right_on='airport')
delay_by_airport_merge['percentage']=(delay_by_airport_merge['total delays']/delay_by_airport_merge['flights']) #% delay column
delay_by_airport_merge = delay_by_airport_merge.head(10)
delay_by_airport_merge #explore new df

In [ ]:
#multiple airport plots

#total flights 
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data= delay_by_airport_merge.sort_values(by='flights'),ax=ax, x=('airport'), y=('flights'))
ax.set_title('Total flights by airport', fontsize=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right") #rotate xlabel
ax.set_xlabel('airport')
ax.set_ylabel("Count")
plt.tight_layout()
plt.show() #plot

#total delay
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data= delay_by_airport_merge.sort_values(by='total delays'),ax=ax, x=('airport'), y=('total delays'))
ax.set_title('Total delays by airport', fontsize=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right") #rotate xlabel
ax.set_xlabel('airport')
ax.set_ylabel('No. of total delays')
plt.tight_layout() #tight layout
plt.show() #plot

#percentage of delays
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data= delay_by_airport_merge.sort_values(by='percentage'),ax=ax, x=('airport'), y=('percentage'))
ax.set_title('Percentage of delays by airport', fontsize=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right") #rotate xlabel
ax.set_xlabel('airport')
ax.set_ylabel('Percentage of delays')
plt.tight_layout() #tight layout
plt.show() #plot

In [ ]:
#average delay in minutes by weekday (departure, arrival, and total delay)

#map days
df['DayOfWeek'] = df['DayOfWeek'].map({1:'Monday',2:'Tuesday',
                                       3:'Wednesday',4:'Thursday',5:'Friday',
                                       6:'Saturday',7:'Sunday'}).astype('str') #weekname column  
#create df with average values
day_mean=df[['DayOfWeek','DepDelay','ArrDelay']].groupby(['DayOfWeek']).mean() #mean delay by day
day_mean['Avg. delay']=(day_mean['DepDelay']+day_mean['ArrDelay'])/2 #avg. delay column
day_mean['Total avg. delay']=(day_mean['DepDelay']+day_mean['ArrDelay']) #tot. delay column
day_mean=day_mean.sort_values(by='Avg. delay',ascending=True) #arrange by avg. delay

#export table to img
day_mean = day_mean.reset_index(drop=False)
day_mean.rename(columns={'DayOfWeek': 'Day of Week', 'DepDelay': 
                         'Avg. Departure Delay','ArrDelay':
                         'Avg. Arrival Delay'}, inplace=True) #rename columns
day_mean_style = day_mean.style.background_gradient() #style table
dfi.export(day_mean_style,"Q1_1_table1.png") #export to png
day_mean_style #explore



In [ ]:
#plot bar chart mean delay delays total

#reset plot style for next to plots (to match color of bars)
sns.reset_orig()
sns.set_style("whitegrid")

f, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data = day_mean.sort_values(by='Total avg. delay'), 
            ax=ax, alpha=.8, x=('Day of Week'), y=('Total avg. delay'))

ax.set_title('Average delay of weekday (min.)', fontsize=30, fontname="Monospace", alpha=.8) #title
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right", fontsize = 15) #rotate xlabel
ax.set_ylabel('Avg. delay (min)', fontsize = 20)
ax.set_xlabel(xlabel=None) #remove obvioius xlabel
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout rect
plt.savefig('Q1_1_plot1.png', bbox_inches='tight') #save png
plt.show() #plot

In [ ]:
#total flights/delays and percentage of delays by weekday (total delay)

#df with total no. of delay occrurences by weekday in percentage
day_total1=df[['DayOfWeek','delay']]
day_total1=df[['DayOfWeek','delay']].groupby(['DayOfWeek']).count() #group by day and count total per day.
day_total1['Flights(%)']=(day_total1['delay']/14007604)*100 #delay delays/ total flights in dataset
day_total1.rename(columns={'delay': 'Flights'}, inplace=True)

#total delays by weekday
day_total2 = df[['DayOfWeek','DepDelay','ArrDelay']]
day_total2 = (day_total2[(day_total2['DepDelay']>0) | (day_total2['ArrDelay']>0)])
day_total2.rename(columns={'DepDelay': 'Delays'}, inplace=True)
day_total2=day_total2.groupby(['DayOfWeek']).count()
day_total2.drop(['ArrDelay'], inplace=True, axis=1) #drop columns
day_total2

#merge on DayofWeek
day_total = pd.merge(day_total1, day_total2,  how='inner', left_on=['DayOfWeek'], 
                     right_on = ['DayOfWeek'])
day_total['Delays (%)'] = (day_total['Delays']/day_total['Flights']) #create delay in percentage column
day_total = day_total.sort_values(by='Delays (%)',ascending=True)#sort values

#prettify table
day_total = day_total.reset_index(drop=False)
day_total.rename(columns={'DayOfWeek': 'Day of Week'}, inplace=True)
day_total_style = day_total.style.background_gradient()
dfi.export(day_total_style,"Q1_1_table2.png")

day_total_style

In [ ]:
my_palette = sns.color_palette("colorblind")


In [ ]:
#plot bar chart mean delay delays total
f, ax = plt.subplots(figsize=(10, 6))

palette=['C1','C0','C2','C4','C3','C5','C6','c8'] #manually set day-color to match previous plot 

sns.barplot(data = day_total[['Day of Week','Delays (%)']],ax=ax,alpha=.8, palette=palette, 
            x=('Day of Week'), y=('Delays (%)'))

ax.set_title('Average delay of weekday (%)', fontsize=30, fontname="Monospace", alpha=.8)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right", fontsize = 15) #rotate xlabel
ax.set_ylabel('Percentage', fontsize = 20)
ax.set_xlabel(xlabel=None) #remove obvioius xlabel
plt.gca().set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()]) 
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout
plt.savefig('Q1_1_plot2.png', bbox_inches='tight')
plt.show() #plot

In [ ]:
#set plotting style for following plots in notebook
sns.reset_orig()
plt.style.use('seaborn-colorblind')
sns.set_style("whitegrid")

In [ ]:
#check for missing outliers
print(df['ArrTime'].isna().sum())
print(df['ArrTime'].value_counts())

print(df['DepTime'].isna().sum())
df['DepTime'].value_counts()

In [ ]:
#create bins with time of the day for ArrTime and DepTime
def condition1(i): 
    if i >= 0 and i <=600: return 'night'
    if i > 600 and i <=1200: return 'before noon'
    if i > 1200 and i <=1800: return 'afternoon'
    if i > 1800 and i <=2400: return 'evening'
    else: return 'other' #group invalid time entries (i.e 2600 in military time is not valid)

df['arr_time_bin']=df['ArrTime'].apply(condition1) #apply conditions to departure
df['dep_time_bin']=df['DepTime'].apply(condition1) #apply conditions to arrival

#choose related columns for df
time_day = df[['ArrTime','CRSArrTime','ArrDelay','DepTime','CRSDepTime',
               'DepDelay','delay','arr_time_bin', 'dep_time_bin']]
time_day =  time_day[(time_day['dep_time_bin'] != 'other') & 
                     (time_day['arr_time_bin'] != 'other')] #exclude 'none' = invalid time
time_day #explore

In [ ]:
#explore time bins for departure and arrival
print(time_day.arr_time_bin.value_counts())
print(time_day.dep_time_bin.value_counts())

In [ ]:
#create time of day table for flights, delays, and delays %.

#total flights departure
time_day_dep=time_day[['dep_time_bin','delay']] #select columns
time_day_dep=time_day[['dep_time_bin','delay']].groupby(['dep_time_bin']).count() #group by day and count total per day.
time_day_dep = time_day_dep.reset_index(drop=False) #reset index
time_day_dep.rename(columns={'delay': 'Flights','dep_time_bin':'Time of day'}, inplace=True) #rename

#total flights arrival
time_day_arr=time_day[['arr_time_bin','delay']] #select columns
time_day_arr=time_day[['arr_time_bin','delay']].groupby(['arr_time_bin']).count() #group by day and count total per day.
time_day_arr = time_day_arr.reset_index(drop=False) #reset index
time_day_arr.rename(columns={'delay': 'Flights','arr_time_bin': 'Time of day'}, inplace=True)  #rename

#merge departure and arrival flights
time_day_tot_comb = pd.merge(time_day_dep, time_day_arr,  how='left', 
                             left_on=['Time of day'], right_on = ['Time of day'])
time_day_tot_comb['Flights'] = time_day_tot_comb['Flights_x']+time_day_tot_comb['Flights_y']
time_day_tot_comb = time_day_tot_comb[['Time of day','Flights']]

#create delay column for departure2
time_day_dep2 = time_day[['dep_time_bin','DepDelay']]
time_day_dep2 = (time_day_dep2[(time_day_dep2['DepDelay']>0)])
time_day_dep2 = time_day_dep2.groupby(['dep_time_bin']).count()
time_day_dep2 = time_day_dep2.reset_index(drop=False)
time_day_dep2.rename(columns={'DepDelay': 'Total dep. delay','dep_time_bin':
                              'Time of day'}, inplace=True)

#create delay column for arrival2
time_day_arr2 = time_day[['arr_time_bin','ArrDelay']]
time_day_arr2 = (time_day_arr2[(time_day_arr2['ArrDelay']>0)])
time_day_arr2 = time_day_arr2.groupby(['arr_time_bin']).count()
time_day_arr2 = time_day_arr2.reset_index(drop=False)
time_day_arr2.rename(columns={'ArrDelay': 'Total arr. delay','arr_time_bin':
                              'Time of day'}, inplace=True)


#merge departure and arrival delays
time_day_tot_comb2 = pd.merge(time_day_dep2, time_day_arr2,  how='left', 
                              left_on=['Time of day'], right_on = ['Time of day'])
time_day_tot_comb2['Delays'] = time_day_tot_comb2['Total dep. delay']+time_day_tot_comb2['Total arr. delay']

#merge all df's
time_day_df = pd.merge(time_day_tot_comb, time_day_tot_comb2,  how='left', 
                       left_on=['Time of day'], right_on = ['Time of day'])
time_day_df['Delayed (%)'] = (time_day_df['Delays']/time_day_df['Flights'])*100
time_day_df = time_day_df.sort_values(by='Delayed (%)',ascending=True)

time_day_df

In [ ]:
#selct varibales and prettify table
time_day_df = time_day_df[['Time of day','Flights','Delays','Delayed (%)']]
time_day_df.rename(columns={'dep_time_bin': 'Time of Day'}, inplace=True)
time_day_df_style = time_day_df.style.background_gradient()
dfi.export(time_day_df_style,"Q1_2_table1.png") #to png
time_day_df_style

In [ ]:
#plot bar chart total delay delays total for day
f, ax = plt.subplots(figsize=(10, 5))
sns.barplot(data = time_day_df.sort_values(by='Delayed (%)'),ax=ax,alpha=.8, x=('Time of day'), y=('Delayed (%)'))
#labels
ax.set_title('Percentage of delays by time of day', fontsize=30, fontname="Monospace", alpha=.8)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right", fontsize = 15) #rotate xlabel
ax.set_xlabel(xlabel=None) #remove obvioius xlabel
ax.set_ylabel('Percentage', fontsize = 16)
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout
plt.savefig('Q1_2_plot1.png', bbox_inches='tight')
plt.show() #plot

In [ ]:
#mean of arrival delay
time_day_arr_mean = time_day[['ArrDelay','arr_time_bin']].groupby(['arr_time_bin']).mean().reset_index(drop=False)
time_day_arr_mean.rename(columns={'arr_time_bin': 'Time of day','ArrDelay':'Avg. Arrival Delay'}, inplace=True)

#mean of departure delay
time_day_dep_mean = time_day[['DepDelay','dep_time_bin']].groupby(['dep_time_bin']).mean().reset_index(drop=False)
time_day_dep_mean.rename(columns={'dep_time_bin': 'Time of day','DepDelay':'Avg. Departure Delay'}, inplace=True)

# merge and create total column
time_day_mean_comb = pd.merge(time_day_dep_mean, time_day_arr_mean,  how='left', left_on=['Time of day'], right_on = ['Time of day'])
time_day_mean_comb['Total avg. Delays'] = (time_day_mean_comb['Avg. Departure Delay'] + time_day_mean_comb['Avg. Arrival Delay'])
time_day_mean_comb=time_day_mean_comb.sort_values(by='Total avg. Delays',ascending=True)

#style table
time_day_mean_comb_style = time_day_mean_comb.style.background_gradient()
dfi.export(time_day_mean_comb_style,"Q1_2_table2.png")
time_day_mean_comb_style

In [ ]:
#plot bar chart total delay delays total for day
f, ax = plt.subplots(figsize=(10, 5))

palette=['C0','C2','C3','C1'] #set color to match days in previous plot

sns.barplot(data = time_day_mean_comb.sort_values(by='Total avg. Delays'),ax=ax,alpha=.8, x=('Time of day'), y=('Total avg. Delays'),palette=palette)

ax.set_title('Average delay by time of day', fontsize=30, fontname="Monospace", alpha=.8)
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right", fontsize = 15) #rotate xlabel
ax.set_xlabel(xlabel=None) #remove obvioius xlabel
ax.set_ylabel('Minutes', fontsize = 16)
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout
plt.savefig('Q1_2_plot2.png', bbox_inches='tight')
plt.show() #plot

In [ ]:
#map months
df['Month'] = df['Month'].map({1:'January',2:'February',3:'March',4:'April',5:'May',
                               6:'June',7:'July',8:'August',9:'September',10:'October',
                               11:'November',12:'December'})
#df grouped by mean of month 
month_mean=df[['Month','DepDelay','ArrDelay']].groupby(['Month']).mean().reset_index(drop=False)
month_mean['Total Avg. Delay']=(month_mean['DepDelay']+month_mean['ArrDelay']) #mean of the delays
month_mean.rename(columns={'DepDelay': 'Departure delay','ArrDelay':'Arrival Delay'}, inplace=True)
month_mean = month_mean.sort_values(by='Total Avg. Delay',ascending=True)
month_mean_style = month_mean.style.background_gradient() #style
dfi.export(month_mean_style,"Q1_3_table1.png") #save to png
month_mean_style #explore

In [ ]:
#total no. of delay occrurences by weekday in percentage
#month_total1=df[['Month','delay']]
month_total1=df[['Month','delay']].groupby(['Month']).count() #group by day and count total per day.
month_total1['Flights(%)']=(month_total1['delay']/14007604)*100 #delay delays/ total flights in dataset
month_total1.rename(columns={'delay': 'Flights'}, inplace=True)

#total delays by month
month_total2 = df[['Month','DepDelay','ArrDelay']]
month_total2 = (month_total2[(month_total2['DepDelay']>0) | (month_total2['ArrDelay']>0)])
month_total2.rename(columns={'DepDelay': 'Delays'}, inplace=True)
month_total2=month_total2.groupby(['Month']).count()
month_total2.drop(['ArrDelay'], inplace=True, axis=1) #drop columns

#merge on DayofWeek
month_total = pd.merge(month_total1, month_total2,  how='inner', left_on=['Month'], right_on = ['Month'])

#create delay in percentage column
month_total['Delayed (%)'] = month_total['Delays']/month_total['Flights']*100


#sort values
month_total = month_total.sort_values(by='Delayed (%)',ascending=True)

#export cell (dataframe_image library)
month_total = month_total.reset_index(drop=False)
month_total_style = month_total.style.background_gradient()
dfi.export(month_total_style,"Q1_3_table2.png")
month_total_style

In [ ]:
#piechart of average delay month

#data prep
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 'November', 'December'] #list order
month_mean["Months"] = pd.Categorical(month_mean["Month"], categories=month_order) #map order
month_mean = month_mean.sort_values(by='Months') #sort

#pie chart
f, ax = plt.subplots(figsize=(10, 8))
explode = [0.005, 0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.15,0.005,0.005,0.005] #explode september
#create pie chart
plt.pie(month_mean['Total Avg. Delay'], labels= month_mean['Months'], colors = sns.color_palette('pastel')[2:13], 
        explode = explode, autopct='%.2f%%') #plot
plt.title('Delay dist. by month (%)', fontsize=18, fontname="Monospace", alpha=.8) #title
plt.savefig('Q1_3_plot1.png', bbox_inches='tight') #save to png
plt.show() #plot

In [ ]:
season_total = month_total

#season total with function
def label1 (row):
   if row['Month'] == 'March' or row['Month'] == 'April' or row['Month'] == 'May':
      return 'Spring'
   if row['Month'] == 'June' or row['Month'] == 'July' or row['Month'] == 'August':
      return 'Summer'
   if row['Month'] == 'September' or row['Month'] == 'October' or row['Month'] == 'November':
      return 'Autumn'
   if row['Month'] == 'January' or row['Month'] == 'February' or row['Month'] == 'December':
      return 'Winter'
   return 'Other'
season_total['Month']=season_total.apply(lambda row: label1(row), axis=1)

#group and calculate season total df
season_total = season_total.groupby(['Month']).sum().reset_index(drop=False)
season_total['Delayed (%)'] = season_total['Delayed (%)']/3
season_total.rename(columns={'Month': 'Season'}, inplace=True)
season_total_style = season_total.style.background_gradient()
dfi.export(season_total_style,"Q1_3_table3.png")
season_total_style

In [ ]:
#same for mean:
season_mean = month_mean
def label2 (row):
   if row['Month'] == 'March' or row['Month'] == 'April' or row['Month'] == 'May':
      return 'Spring'
   if row['Month'] == 'June' or row['Month'] == 'July' or row['Month'] == 'August':
      return 'Summer'
   if row['Month'] == 'September' or row['Month'] == 'October' or row['Month'] == 'November':
      return 'Autumn'
   if row['Month'] == 'January' or row['Month'] == 'February' or row['Month'] == 'December':
      return 'Winter'
   return 'Other'
season_mean['Month']=season_mean.apply(lambda row: label2(row), axis=1)

#group and calculate season total df
season_mean = season_mean.groupby(['Month']).sum().reset_index(drop=False)
season_mean.rename(columns={'Month': 'Season','Departure delay':'Dep. delay (min)',
                            'Arrival Delay':'Arr. delay (min)'}, inplace=True)
season_mean_style = season_mean.style.background_gradient()
dfi.export(season_mean_style,"Q1_3_table4.png")
season_mean_style

In [ ]:
#combine plane dataframe with planedata
df_airplanes= df.merge(airplanes[['tailnum','year']],how='left',left_on='TailNum',
                       right_on='tailnum') #join
df_airplanes=df_airplanes[df_airplanes['year'].isnull()==False] #exlude

#rename year from airplanes (to avoid confusion)
df_airplanes.rename(columns={'year': 'plane_year'}, inplace=True)
df_airplanes #explore 


In [ ]:
df_airplanes.isna().sum()

In [ ]:
#NA's in plane_year is zero, but a closer look reveals the some values = None(a missing value)
df_airplanes.value_counts('plane_year').head(15)

In [ ]:
#exclude the 'None' values
df_airplanes = df_airplanes[df_airplanes.plane_year != 'None']
df_airplanes.value_counts('plane_year').head(15) #verify

In [ ]:
df_airplanes['plane_year'].dtype #check dtype

In [ ]:
df_airplanes['plane_year'] = df_airplanes['plane_year'].astype(int) #change to integer
df_airplanes['plane_year'].dtype #verify

In [ ]:
#categorise the planes either as old or new by manifacturing year
bins = [1920,1987,2022]
labels=['old','new']
df_airplanes['plane_condition'] = pd.cut(df_airplanes['plane_year'], bins=bins, labels=labels)

#create a delayed boolean column 
df_airplanes['Delayed_bool'] = (df_airplanes['DepDelay']>0) | (df_airplanes['ArrDelay']>0)
df_airplanes['Delayed_bool'] = df_airplanes['Delayed_bool']*1

df_airplanes_bin=df_airplanes[['plane_condition','plane_year','delay','Delayed_bool', 'ArrDelay','DepDelay']]
df_airplanes_bin

In [ ]:
df_airplanes_bin.isna().sum()

In [ ]:
#the new NA's is must be occuring due to the year bins, let's adress these:

#check for outlier values
print(df_airplanes_bin['plane_year'].min())
print(df_airplanes_bin['plane_year'].max())

In [ ]:
#0 is definetely not a correct value for manufacturing year, lets drop theese.

df_airplanes_bin = df_airplanes_bin[df_airplanes_bin['plane_year'] > 1900] #exclude 0
print(df_airplanes_bin['plane_year'].min())
print(df_airplanes_bin['plane_year'].max())

In [ ]:
#Year 1956 seems more likely to be the manifacturing year of the oldest plane.
#NA's in df_airplanes should now be zero.
df_airplanes_bin.isna().sum()

In [ ]:
#Total no. of old and new planes
df_airplanes_bin.plane_condition.value_counts()


In [ ]:
#total no. of delay occrurences by weekday in percentage
df_airplanes_bin1=df_airplanes_bin[['plane_condition','delay']].groupby(['plane_condition']).count() #group by day and count total per day.
df_airplanes_bin1['Flights(%)']=(df_airplanes_bin1['delay']/(10100221+1232687) )*100 #divide with total flights by old and new planes
df_airplanes_bin1.rename(columns={'delay': 'Flights'}, inplace=True)
df_airplanes_bin1

#total delays by weekday
df_airplanes_bin2 = df_airplanes_bin[['plane_condition','Delayed_bool']].groupby(['plane_condition']).sum()
df_airplanes_bin2.rename(columns={'Delayed_bool': 'Delays'}, inplace=True)

df_airplanes_tab = pd.merge(df_airplanes_bin1, df_airplanes_bin2,  how='inner', left_on=['plane_condition'], right_on = ['plane_condition'])

#create delay in percentage column
df_airplanes_tab['Delayed (%)'] = df_airplanes_tab['Delays']/df_airplanes_tab['Flights']*100

#sort values
df_airplanes_tab = df_airplanes_tab.sort_values(by='Delayed (%)',ascending=True)

#export cell (dataframe_image library)
df_airplanes_tab = df_airplanes_tab.reset_index(drop=False)
df_airplanes_tab.rename(columns={'plane_condition': 'Plane Condition'}, inplace=True)
df_airplanes_tab_style = df_airplanes_tab.style.background_gradient()
dfi.export(df_airplanes_tab_style,"Q2_table1.png")
df_airplanes_tab_style

In [ ]:
#airplane mean by manu. year
df_airplanes_mean=df_airplanes_bin[['plane_condition',
                                    'DepDelay','ArrDelay','delay']].groupby(['plane_condition']).mean().reset_index(drop=False) #group
df_airplanes_mean.rename(columns={'delay': 
                                  'Total avg. delay','ArrDelay':'Avg. Arrival Delay',
                                  'DepDelay':'Avg. Departure Delay','plane_condition': 'Plane Condition'}, inplace=True) #rename
df_airplanes_mean = df_airplanes_mean.sort_values(by='Total avg. delay',ascending=True) #sort
df_airplanes_mean_style = df_airplanes_mean.style.background_gradient()
dfi.export(df_airplanes_mean_style,"Q2_table2.png")
df_airplanes_mean_style

In [ ]:
#merge the tables

df_airplanes_comb = pd.merge(df_airplanes_tab,df_airplanes_mean,  how='inner', left_on=['Plane Condition'], right_on = ['Plane Condition'])

df_airplanes_comb_style = df_airplanes_comb.style.background_gradient() #style
dfi.export(df_airplanes_comb_style,"Q2_table3.png") #export to png
df_airplanes_comb_style #explore

In [ ]:
#plot mean delay

df_airplanes_mean = df_airplanes_mean.reset_index(drop=False)

#plot barchart
f, ax = plt.subplots(figsize=(10, 7))

df_airplanes_mean.plot(kind = 'bar',ax=ax, x='Plane Condition',y='Total avg. delay', color=['indianred', 'olivedrab'],legend=None)

#set labels
ax.set_title("Mean delay: Old vs. New", fontsize=17) #title
plt.ylabel("Mean delay") #ylabel
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha="center",fontsize=14 ) #rotate xlabel
#plt.tight_layout() #tight layout
plt.show() #plot

In [ ]:
#plot data prep
condition_data=[20.436394,20.647952] 
condition_dec = pd.Series(index = ["New planes", "Old planes"], data=condition_data) #create series
percentage = condition_dec.div(condition_dec.sum()).mul(100).round(2) #calc. the percentage and round 2 dec



#barplot 
ax = condition_dec.plot(kind="bar", alpha=0.7, figsize=(10, 8), color=palette)
#set labels inside the bars
labels = [f'{condition_dec} - {percentage[condition_dec]}%' for condition_dec in condition_dec.index]
for label, a in zip(labels, ax.patches):
    left, bottom, width, height = a.get_bbox().bounds
    ax.annotate(label, xy=(left+width/2, bottom+height/2), ha='center', va='center', rotation=90, fontsize=28)
#labels
plt.ylabel("Minutes", fontsize=25) #ylabel
plt.xlabel("Plane condition", fontsize=25) #xlabel
plt.title("Mean time of delays for planes", fontsize=30, fontname="Monospace", alpha=.8) #title
plt.xticks([]) #empty xticks
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout (rectangular)
plt.savefig('Q2_plot1.png', bbox_inches='tight') #save img
plt.show()

#same for total delays by plane:
#data prep
condition_data2=[55.906308,57.950508] 
condition_dec2 = pd.Series(index = ["New planes - 55.91", "Old planes - 57.95"], data=condition_data2) #create series

#barplot 
ax = condition_dec2.plot(kind="bar", alpha=0.7, figsize=(10, 8), color=palette)
#set labels inside the bars
labels = [f'{condition_dec2}%' for condition_dec2 in condition_dec2.index]
for label, a in zip(labels, ax.patches):
    left, bottom, width, height = a.get_bbox().bounds
    ax.annotate(label, xy=(left+width/2, bottom+height/2), ha='center', va='center', rotation=90, fontsize=28)
#labels
plt.ylabel("Percentage", fontsize=25) #ylabel
plt.xlabel("Plane condition", fontsize=25) #xlabel
plt.title("Percentage of delayed flight", fontsize=30, fontname="Monospace", alpha=.8) #title
plt.xticks([]) #empty xticks
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout (rectangular)
plt.savefig('Q2_plot2.png', bbox_inches='tight') #save img
plt.show()

In [ ]:
#chi-squared test

#create a contingency table
airplane_contingency  = pd.crosstab(df_airplanes['plane_condition'],
                            df_airplanes['Delayed_bool'],
                           margins=True, margins_name="Total") #margins=true adds total column
airplane_contingency #explore table

#calculate relation
chi2, p, expected,dof  = chi2_contingency(airplane_contingency)
print("H0: Old planes does not suffer more delay VS. H1:Old planes suffer more delay ")
print(f"P Value:            {p:.4f}")
if p >=0.05:
  print("We reject the null hypotheses at a 5% significance level") #% significance level
else:
  print("We FAIL reject the null hypotheses at a 5% significance level")

#we also test at a 10% significance level due to the close mean values
if p >=0.10:
  print("We reject the null hypotheses at a 10% significance level") #10% significance level
else:
  print("We also FAIL reject the null hypotheses at a 10% significance level")

In [ ]:
#categorise the planes either as old or new by manifacturing year
bins = [0,1987,1997,2022]
labels=['1956-1987(old)','1987-1997(mid)','1997-2007(new)']
df_airplanes['plane_condition2'] = pd.cut(df_airplanes['plane_year'], bins=bins, labels=labels)
df_airplanes_bin2=df_airplanes[['plane_condition2','plane_year','delay','Delayed_bool', 
                                'ArrDelay','DepDelay']]

#group by condition and mean
df_airplanes_mean2=df_airplanes_bin2[['plane_condition2',
                                      'DepDelay','ArrDelay',
                                      'delay']].groupby(['plane_condition2']).mean().reset_index(drop=False)
df_airplanes_mean2.rename(columns={'plane_condition2':
                                   'Plane Condition','delay': 'Total avg. delay','ArrDelay':
                                   'Avg. Arrival Delay','DepDelay':
                                   'Avg. Departure Delay'}, inplace=True)
df_airplanes_mean2 = df_airplanes_mean2.sort_values(by='Total avg. delay',ascending=True)
df_airplanes_mean2_style = df_airplanes_mean2.style.background_gradient() #style
dfi.export(df_airplanes_mean2_style,"Q2_table4.png") #export to png
df_airplanes_mean2_style #explore

In [ ]:
#total no. of delay occrurences by weekday in percentage
df_airplanes_bin2_1=df_airplanes_bin2[['plane_condition2','delay']].groupby(['plane_condition2']).count() #group by day and count total per day.
df_airplanes_bin2_1.rename(columns={'delay': 'Flights'}, inplace=True)
df_airplanes_bin2_1['Flights(%)']=(df_airplanes_bin2_1['Flights']/11332781 )*100 #divide with total flights by old and new planes


#total delays by weekday
df_airplanes_bin2_2 = df_airplanes_bin2[['plane_condition2','Delayed_bool']].groupby(['plane_condition2']).sum()
df_airplanes_bin2_2.rename(columns={'Delayed_bool': 'Delays'}, inplace=True)

#merge on plane condition
df_airplanes_bin2_tab = pd.merge(df_airplanes_bin2_1, df_airplanes_bin2_2,  how='inner', left_on=['plane_condition2'], right_on = ['plane_condition2'])

#create delay in percentage column
df_airplanes_bin2_tab['Delayed (%)'] = df_airplanes_bin2_tab['Delays']/df_airplanes_bin2_tab['Flights']*100

#create delay in percentage column
df_airplanes_bin2_tab['Delayed (%)'] = df_airplanes_bin2_tab['Delays']/df_airplanes_bin2_tab['Flights']*100
df_airplanes_bin2_tab=df_airplanes_bin2_tab.reset_index(drop=False)
df_airplanes_bin2_tab.rename(columns={'plane_condition2': 'Plane Condition'}, inplace=True)


df_airplanes_bin_comb = pd.merge(df_airplanes_bin2_tab,df_airplanes_mean2,  how='inner', left_on=['Plane Condition'], right_on = ['Plane Condition'])

df_airplanes_bin_comb_style = df_airplanes_bin_comb.style.background_gradient() #style
dfi.export(df_airplanes_bin_comb_style,"Q2_table5.png") #export to png
df_airplanes_bin_comb_style #explore

In [ ]:
df_airplanes_mean2 = df_airplanes_mean2.reset_index(drop=False)

#plot barchart
f, ax = plt.subplots(figsize=(10, 7))

df_airplanes_mean2.plot(kind = 'bar',ax=ax, x='Plane Condition',y='Total avg. delay',color=['indianred', 'olivedrab', 'blue'],legend=None)

#set labels
ax.set_title("Mean delay: Old vs. New", fontsize=17) #title
plt.ylabel("Mean delay") #ylabel
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha="center",fontsize=14 ) #rotate xlabel
#plt.tight_layout() #tight layout
plt.show() #plot

In [ ]:
#prettify the plot for report
condition_data3=[20.127357,20.647952,21.079243] 
condition_dec3 = pd.Series(index = ["New planes", "Old planes",'Mid planes'], data=condition_data3) #create series
percentage3 = condition_dec3.div(condition_dec3.sum()).mul(100).round(2) #calc. the percentage and round 2 dec

#barplot 
ax = condition_dec3.plot(kind="bar", alpha=0.7, figsize=(10, 8), color=palette)
#set labels inside the bars
labels = [f'{condition_dec3} - {percentage3[condition_dec3]}%' for condition_dec3 in condition_dec3.index]
for label, a in zip(labels, ax.patches):
    left, bottom, width, height = a.get_bbox().bounds
    ax.annotate(label, xy=(left+width/2, bottom+height/2), ha='center', va='center', rotation=90, fontsize=28)
#labels
plt.ylabel("Minutes", fontsize=25) #ylabel
plt.xlabel("Plane condition", fontsize=25) #xlabel
plt.title("Mean time of delays for planes", fontsize=30, fontname="Monospace", alpha=.8) #title
plt.xticks([]) #empty xticks
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout (rectangular)
plt.savefig('Q2_plot3.png', bbox_inches='tight') #save img
plt.show()


#prettify the plot for report
condition_data4=[55.35,57.95,57.07] 
condition_dec4 = pd.Series(index = ["New planes - 55.35", "Old planes - 57.95",'Mid planes - 57.07'], data=condition_data3) #create series

#barplot 
ax = condition_dec3.plot(kind="bar", alpha=0.7, figsize=(10, 8), color=palette)
#set labels inside the bars
labels = [f'{condition_dec4}%' for condition_dec4 in condition_dec4.index]
for label, a in zip(labels, ax.patches):
    left, bottom, width, height = a.get_bbox().bounds
    ax.annotate(label, xy=(left+width/2, bottom+height/2), ha='center', va='center', rotation=90, fontsize=28)
#labels
plt.ylabel("Percentage", fontsize=25) #ylabel
plt.xlabel("Plane condition", fontsize=25) #xlabel
plt.title("Percentage of delayed flight", fontsize=30, fontname="Monospace", alpha=.8) #title
plt.xticks([]) #empty xticks
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout (rectangular)
plt.savefig('Q2_plot4.png', bbox_inches='tight') #save img
plt.show()

In [ ]:
#load data for a ten year period, same encoding used for all.
df_97 = pd.read_csv("1997.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_98 = pd.read_csv("1998.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_99 = pd.read_csv("1999.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_00 = pd.read_csv("2000.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_01 = pd.read_csv("2001.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_02 = pd.read_csv("2002.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_03 = pd.read_csv("2003.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_04 = pd.read_csv("2004.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_05 = pd.read_csv("2005.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')
df_06 = pd.read_csv("2006.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1') 
df_07 = pd.read_csv("2007.csv", usecols = ["Origin", "Dest", "Year"], encoding='latin-1')  


In [ ]:
#group data and rename the year column to avoid confusion
df_97 = df_97.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_97.rename(columns={'Year': '1997'}, inplace=True) #year 1997
df_98 = df_98.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_98.rename(columns={'Year': '1998'}, inplace=True)#year 1998
df_99 = df_99.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_99.rename(columns={'Year': '1999'}, inplace=True)#year 1999
df_00 = df_00.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_00.rename(columns={'Year': '2000'}, inplace=True)#year 2000
df_01 = df_01.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_01.rename(columns={'Year': '2001'}, inplace=True)#year 2001
df_02 = df_02.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_02.rename(columns={'Year': '2002'}, inplace=True)#year 2002
df_03 = df_03.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_03.rename(columns={'Year': '2003'}, inplace=True)#year 2003
df_04 = df_04.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_04.rename(columns={'Year': '2004'}, inplace=True)#year 2004
df_05 = df_05.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_05.rename(columns={'Year': '2005'}, inplace=True)#year 2005
df_06 = df_06.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_06.rename(columns={'Year': '2006'}, inplace=True)#year 2006
df_07 = df_07.groupby(['Origin','Dest']).count().sort_values(by=['Year'], ascending=False)
df_07.rename(columns={'Year': '2007'}, inplace=True)#year 2007

In [ ]:
#specify dataframes to merge
data_frames = [df_97, df_98, df_99, df_00, df_01, df_02, df_03, df_04, df_05, df_06, df_07]

#merge to one df
df_location = reduce(lambda  left,right: pd.merge(left,right,on=['Origin','Dest'],
                                            how='left'), data_frames)
df_location #explore

In [ ]:
#see missing values
df_location.isna().sum()

In [ ]:
#replace with NaN with 0 because it represents a missing entry (i.e. no arrival or departure from airport).
df_location = df_location.replace(np.nan, 0)

#verify
df_location.isnull().sum() 

In [ ]:
#create total column
df_location['Total'] = df_location.sum(axis=1)
df_location=df_location.sort_values(by=['Total'], ascending=False).reset_index(drop=False)

df_location

In [ ]:
#select top 20 locations
top20 = df_location.loc[0:19]

def label3 (row):
   if row['Origin'] == 'LAX' and row['Dest'] == 'LAS':
      return 'LAX&LAS'
   if row['Origin'] == 'LAS' and row['Dest'] == 'LAX':
      return 'LAX&LAS'
   if row['Origin'] == 'PHX' and row['Dest'] == 'LAX':
      return 'PHX&LAX'
   if row['Origin'] == 'LAX' and row['Dest'] == 'PHX':
      return 'PHX&LAX'
   if row['Origin'] == 'SFO' and row['Dest'] == 'LAX':
      return 'SFO&LAX'
   if row['Origin'] == 'LAX' and row['Dest'] == 'SFO':
      return 'SFO&LAX'
   if row['Origin'] == 'ORD' and row['Dest'] == 'MSP':
      return 'ORD&MSP'
   if row['Origin'] == 'MSP' and row['Dest'] == 'ORD':
      return 'ORD&MSP'
   if row['Origin'] == 'PHX' and row['Dest'] == 'LAS':
      return 'PHX&LAS'
   if row['Origin'] == 'LAS' and row['Dest'] == 'PHX':
      return 'PHX&LAS'
   if row['Origin'] == 'LGA' and row['Dest'] == 'ORD':
      return 'LGA&ORD'
   if row['Origin'] == 'ORD' and row['Dest'] == 'LGA':
      return 'LGA&ORD'
   if row['Origin'] == 'HOU' and row['Dest'] == 'DAL':
      return 'HOU&DAL'
   if row['Origin'] == 'DAL' and row['Dest'] == 'HOU':
      return 'HOU&DAL'
   if row['Origin'] == 'OAK' and row['Dest'] == 'LAX':
      return 'OAK&LAX'
   if row['Origin'] == 'LAX' and row['Dest'] == 'OAK':
      return 'OAK&LAX'
   if row['Origin'] == 'EWR' and row['Dest'] == 'ORD':
      return 'EWR&ORD'
   if row['Origin'] == 'ORD' and row['Dest'] == 'EWR':
      return 'EWR&ORD'
   if row['Origin'] == 'BOS' and row['Dest'] == 'LGA':
      return 'BOS&LGA'
   if row['Origin'] == 'LGA' and row['Dest'] == 'BOS':
      return 'BOS&LGA'
   return 'Other'

top20['connection']=top20.apply(lambda row: label3(row), axis=1)

In [ ]:
#group to top10 connections/routes
top10 = top20.groupby("connection").sum().reset_index(drop=False) #sum 
top10.rename(columns={'connection': 'Route'}, inplace=True) #rename
top10 = top10.sort_values(by=['Total'], ascending=False) #arrange by total column
top10 = top10.drop('Total', 1) #drop total column again.

#convert to integer
for col in top10.columns:
    if top10[col].dtype == np.float:
        top10[col] = top10[col].astype(int)

top10_Style = top10.style.background_gradient() #prettify
dfi.export(top10_Style,"Q3_table1.png") #export to image
top10_Style #explore

In [ ]:
#plot 11 year stacked
f, ax = plt.subplots(figsize=(16, 8))


colors=['blue','red','green','gold','peru','olive','cyan',
        'crimson','purple','steelblue','orange'] #colour
top10.set_index('Route').plot(kind='bar', stacked=True,ax=ax,color=colors) #plot
ax.legend(loc='upper right',title='Year', bbox_to_anchor=(1, 1),
          ncol=4, fancybox=True, shadow=True) #legend
plt.title('Location changes over time', fontsize=18) #title
plt.ylabel('Number of flights') #ylabel
plt.show()

Plot of years that stand out a bit (including start and end year)

In [ ]:
#data prep
start_to_end = top10[['Route','1997','2001','2002','2007']]

#reset plot style for next to plots (to match color of bars)
sns.reset_orig()
sns.set_style("whitegrid")

#plot
f,ax=plt.subplots(figsize=(12,7))
start_to_end.set_index('Route').plot(kind='bar',stacked=False,ax=ax, alpha=.8)
ax.legend(loc='upper right', title='Year', fontsize=12) #legend
#title and labels,etc:
ax.set_title('Flights between locations (1997-2007)', fontsize=30, fontname="Monospace", alpha=.8)
ax.set_xlabel(xlabel=None) #remove obvioius xlabel
ax.set_xticklabels(ax.get_xticklabels(), rotation=35, ha="right", fontsize = 12) #rotate xlabel
ax.set_ylabel('No. of flights', fontsize=16) #ylab
plt.savefig('Q3_plot1.png', bbox_inches='tight') #export png
plt.show() 

In [ ]:
#set plotting style for following plots in notebook
sns.reset_orig()
plt.style.use('seaborn-colorblind')
sns.set_style("whitegrid")

In [ ]:
#plot total flights over the years

#data prep
top10_tot = top10 
top10_tot.loc['Total']= top10_tot.sum(numeric_only=True,axis=0)
top10_tot = top10_tot.fillna('All Routes')
tot10_tot_data = top10_tot.iloc[10,1:12]
tot10_tot_data = pd.DataFrame(tot10_tot_data)

#plot
f, ax = plt.subplots(figsize=(16, 8))
tot10_tot_data.plot(kind='bar',ax=ax,color=palette)
plt.title('Location changes over time', fontsize=18)
plt.ylabel('Number of flights')
plt.show()

In [ ]:
#Exploration of the drop in 2001
df_2001 = pd.read_csv("2001.csv", usecols = ["Month", "UniqueCarrier"], encoding='latin-1') #read data
df_2001 = df_2001.groupby("Month").count().reset_index(drop=False) #count flights per month
df_2001['Month'] = df_2001['Month'].map({1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}).astype('str') #map names of months

#plot data
f,ax=plt.subplots(figsize=(12,7))
ax.plot(df_2001['Month'], df_2001['UniqueCarrier'], alpha=0.8,linewidth=3)
plt.title("Exploration of drop in 2001", fontsize=30, fontname="Monospace", alpha=.8)
ax.set_xlabel(xlabel=None) #remove obvioius xlabel
plt.ylabel('Total no. of flights', fontsize=16)
plt.savefig('Q3_plot2.png', bbox_inches='tight')
plt.show()


In [ ]:
#create df's
plane_delay = df #df for airplanes
df_airport = df #df for airports

#select columns
plane_delay= plane_delay[['TailNum','Origin','Dest','DepDelay','ArrDelay','Date','DepTime']]
df_airport = df_airport[['Origin','DepDelay','Dest','DepTime','ArrDelay','Date_bin','Month','DayofMonth','dep_hour_int']]

#create column with boolean delay occurence value
def label4 (row): 
    if row <= 0: return 0
    if row > 0: return 1
plane_delay['DepDelay']=plane_delay['DepDelay'].apply(label4) #apply condition
plane_delay['ArrDelay']=plane_delay['ArrDelay'].apply(label4) #apply condition
df_airport['DepDelay']=df_airport['DepDelay'].apply(label4) #apply condition
df_airport['ArrDelay']=df_airport['ArrDelay'].apply(label4) #apply condition

plane_delay #explore df, we use df_airport later.

In [ ]:
plane_delay['TailNum'].value_counts()

In [ ]:
#detect cascading failures
plane_delay_N485HA = plane_delay_N485HA[['TailNum','DepDelay','ArrDelay']] #columns to use

#count consecutive delays

#departure casc. failure: departure delay in A + arrival delay in B + departure delay on next flight C
plane_delay_N485HA['dep_casc']=plane_delay_N485HA['DepDelay']+plane_delay_N485HA['ArrDelay']+plane_delay_N485HA['DepDelay'].shift(1)

#departure casc. failure: arrival delay in A + departure delay in B + arrival delay on next flight C
plane_delay_N485HA['arr_casc']=plane_delay_N485HA['ArrDelay']+plane_delay_N485HA['DepDelay']+plane_delay_N485HA['ArrDelay'].shift(1)

#when a cascading delay has occured, dep_casc or arr_casc is = 3
plane_delay_N485HA2 = plane_delay_N485HA[(plane_delay_N485HA["dep_casc"] == 3.0) | (plane_delay_N485HA["arr_casc"] == 3.0)]

#count occurences
plane_delay_N485HA2 =plane_delay_N485HA2.groupby("TailNum").count().reset_index(drop=False)
plane_delay_N485HA2 = plane_delay_N485HA2[['TailNum','arr_casc']] #sort
plane_delay_N485HA2.rename(columns={'arr_casc': 'Cascading Failures'}, inplace=True) #rename
plane_delay_N485HA2 #explore

In [ ]:
#add a second plane to the table

#create df with N477HA flight
plane_delay_N477HA = plane_delay.loc[plane_delay['TailNum'] == 'N477HA'].sort_values(by=['Date'])
plane_delay_N477HA = plane_delay_N477HA[['TailNum','DepDelay','ArrDelay']]
plane_delay_N477HA['dep_casc']=plane_delay_N477HA['DepDelay']+plane_delay_N477HA['ArrDelay']+plane_delay_N477HA['DepDelay'].shift(1)
plane_delay_N477HA['arr_casc']=plane_delay_N477HA['ArrDelay']+plane_delay_N477HA['DepDelay']+plane_delay_N477HA['ArrDelay'].shift(1)

#df for table
plane_delay_N477HA2 = plane_delay_N477HA[(plane_delay_N477HA["dep_casc"] == 3.0) | (plane_delay_N477HA["arr_casc"] == 3.0)]
plane_delay_N477HA2=plane_delay_N477HA2.groupby("TailNum").count().reset_index(drop=False)
plane_delay_N477HA2 = plane_delay_N477HA2[['TailNum','arr_casc']]
plane_delay_N477HA2.rename(columns={'arr_casc': 'Cascading Failures'}, inplace=True)

two_tails = pd.concat([plane_delay_N485HA2, plane_delay_N477HA2])
two_tails=two_tails.reset_index(drop=True)
two_tails

In [ ]:
#count departure and arrival delays
print(plane_delay_N477HA['DepDelay'][(plane_delay_N477HA["DepDelay"] == 1)].count())
print(plane_delay_N477HA['DepDelay'][(plane_delay_N477HA["DepDelay"] == 1) & (plane_delay_N477HA["ArrDelay"] == 1)].count())

#count departure and arrival delays
print(plane_delay_N485HA['DepDelay'][(plane_delay_N485HA["DepDelay"] == 1)].count())
print(plane_delay_N485HA['DepDelay'][(plane_delay_N485HA["DepDelay"] == 1) & (plane_delay_N485HA["ArrDelay"] == 1)].count())

In [ ]:
#cascading fail / tailnum with initial delay and delay in A and B airports.
N477 = (487/805)*100
N477_B =(487/687)*100
N477_tot =(487/7696)*100

N485 = (564/937)*100
N485_b =(564/788)*100 
N485_tot =(564/7893)*100

print("N477HA with delay in airport A has:", N477, "% cascading delay failures")
print("N477HA with delay in airport A and B has:", N477_B , "% cascading delay failures")
print("N477HA has a cascading delay in totl:", N477_tot , "% of flights")


print("N485HA with delay in airport A has:", N485, "% cascading delay failures")
print("N485HA with delay in airport A and B has:", N485_b, "% cascading delay failures")
print("N477HA has a cascading delay in totl:", N485_tot , "% of flights")

In [ ]:
#create table with all stats
N485HA_1 = [7893]
N485HA_col = ['N485HA','Flights']
N485HA_df = pd.DataFrame(list(zip(N485HA_col,N485HA_1)), columns=[1,2]).explode(2).reset_index(drop=True)
N485HA_df.rename(columns={1:'TailNum' ,2: 'Flights'}, inplace=True)

N477HA_1 = [7696]
N477HA_col = ['N477HA','Flights']
N477HA_df = pd.DataFrame(list(zip(N477HA_col,N477HA_1)), columns=[1,2]).explode(2).reset_index(drop=True)
N477HA_df.rename(columns={1:'TailNum' ,2: 'Flights'}, inplace=True)

two_tails_2 = pd.concat([N485HA_df, N477HA_df],ignore_index=True)

two_tails_3 = pd.merge(two_tails, two_tails_2,  how='inner', left_on=['TailNum'], right_on = ['TailNum'])

two_tails_3['Casc. Failure (%)'] = two_tails_3['Cascading Failures']/two_tails_3['Flights']*100
two_tails_3 = two_tails_3[['TailNum','Flights','Cascading Failures','Casc. Failure (%)']]

#export cell (dataframe_image library)
two_tails_style2 = two_tails_3.style.background_gradient()
dfi.export(two_tails_style2,"Q4_table1.png")
two_tails_style2

In [ ]:
#explore the df we created for airports earlier
df_airport

In [ ]:
#create a time bin column
#choose 3 hours because the flight of interest LAS-LAX is approx. 1h15m, so with delays 
#and a windows to see the potential effects, hence a 3 hour bin seems reasonable.
def label5 (row): 
    if row >= 0 and row <=3: return '00-03'
    if row > 3 and row <=6: return '03-06'
    if row > 6 and row <=9: return '06-09'
    if row > 9 and row <=12: return '09-12'
    if row > 12 and row <=15: return '12-15'
    if row > 15 and row <=18: return '15-18'
    if row > 18 and row <=21: return '18-21'
    if row > 21 and row <=24: return '21-24'

df_airport['time_range_3hour']=df_airport['dep_hour_int'].apply(label5) #apply condition
df_airport

In [ ]:
#select two big airports (LAS VEGAS AND LOS ANGELES)

#Las Vegas
df_LAS = df_airport.loc[(df_airport['Origin'] == 'LAS') & (df_airport['Dest'] == 'LAX')] #only the two
df_LAS = df_LAS[['Origin','DepDelay','Dest','DepTime',
                 'ArrDelay','Date_bin','Month','DayofMonth',
                 'dep_hour_int','time_range_3hour']] #select columns
df_LAS = df_LAS.sort_values('Date_bin') #sort
#add one hour to see how the flight affect following delays in B (dep) and C (arr)
#becauase the flight is approx. 1.15m
df_LAS['Date_bin'] = df_LAS['Date_bin']+timedelta(hours=1) #add one hour to all
#delay in airport A
df_LAS_tot = df_LAS[(df_LAS["ArrDelay"] == 1)]
df_LAS_tot = df_LAS_tot.groupby('Date_bin').count() #the bin with +1H grouped 
df_LAS_tot = df_LAS_tot.reset_index(drop=False)
df_LAS_tot = df_LAS_tot[['Date_bin','DepDelay']]
#rename DepDelay to delay because all counts are where arrdelay is == 1.
df_LAS_tot.rename(columns={'DepDelay':'Delay'}, inplace=True) 

#Los Angeles
df_LAX = df_airport.loc[(df_airport['Origin'] == 'LAX') & (df_airport['Dest'] == 'LAS')] #only the two
df_LAX = df_LAX[['Origin','DepDelay','Dest','DepTime',
                 'ArrDelay','Date_bin','Month','DayofMonth',
                 'dep_hour_int','time_range_3hour']] #columns for LA
df_LAX = df_LAX.sort_values('Date_bin') #sort
#delay for airport B AND C
df_LAX_tot = df_LAX[(df_LAX["DepDelay"] == 1) & (df_LAX["ArrDelay"] == 1)] 
df_LAX_tot = df_LAX_tot.groupby('Date_bin').count() #group and count
df_LAX_tot = df_LAX_tot.reset_index(drop=False)
df_LAX_tot = df_LAX_tot[['Date_bin','DepDelay']] #we dont't need all colums because we only have datapoints where both DepDelay and ArrDelay are true.
df_LAX_tot.rename(columns={'DepDelay':'Delay'}, inplace=True) #rename, delay = both dep and arrdeay has occured.

#merge on the datebin that is matched to reflect cascading failures
las_lax_ABA = pd.merge(df_LAX_tot, df_LAS_tot, on='Date_bin')

#only select where Delay_x(delay in both airport B&C) is equal or more than Delay_y(delay in airport A): 
#otherwise we are just counting the no. of delays in airport A for the particular dataset.
las_lax_ABA = las_lax_ABA[(las_lax_ABA["Delay_x"] >= las_lax_ABA["Delay_y"])] #condition on B&C
las_lax_ABA_sum = las_lax_ABA["Delay_y"].sum()
print("sum Delay_y :", las_lax_ABA["Delay_y"].sum())
las_lax_ABA

In [ ]:
#create table with dummy vaiables
chart_df = df_LAX.reset_index(drop=True).merge(df_LAS.reset_index(drop=True), left_index=True, right_index=True)
print("total flights on route: ",chart_df.shape[0])

#use dummy columns and create a table with cascading delays failures detected
las_lax_chart = chart_df[['Origin_y','Origin_x','Dest_x']] #dumme variable
las_lax_chart.rename(columns={'Origin_y': 'Airport A','Origin_x':'Airport B','Dest_x':
                              'Airport C'}, inplace=True) #rename
las_lax_chart['Cascading failures']=3685 #casc. detected
las_lax_chart=las_lax_chart.head(1)
las_lax_chart['Flights'] = 26033 #total flights
las_lax_chart['Casc. Failure (%)'] = las_lax_chart['Cascading failures']/las_lax_chart['Flights']*100
las_lax_chart_style = las_lax_chart[['Airport A' ,'Airport B','Airport C','Flights',
                                     'Cascading failures','Casc. Failure (%)']] #columns
las_lax_chart_style = las_lax_chart_style.style.background_gradient() #style
dfi.export(las_lax_chart_style,"Q4_table2.png") #export
las_lax_chart_style #explore

In [ ]:
#create table with dummy vaiables
chart_df2 = df_LAX_all.reset_index(drop=True).merge(df_LAS.reset_index(drop=True), left_index=True, right_index=True)
print("total flights on route: ",df_LAX_all.shape[0]) #get total flights no.

#use dummy columns and create a table with cascading delays failures detected
las_all_chart = chart_df[['Origin_y','Origin_x','Dest_x']] #dumme variable
las_all_chart.rename(columns={'Origin_y': 'Airport A','Origin_x':'Airport B','Dest_x':
                              'Airport C'}, inplace=True) #rename
las_all_chart['Cascading failures']=12903 #casc. detected
las_all_chart=las_all_chart.head(1)
las_all_chart['Flights'] = 455728 #total flights from LAX
las_all_chart['Casc. Failure (%)'] = las_all_chart['Cascading failures']/las_all_chart['Flights']*100
las_all_chart = las_all_chart[['Airport A' ,'Airport B','Airport C','Flights',
                                     'Cascading failures','Casc. Failure (%)']] #columns
#set rowname in Airport C
#def label6 (row):
 #  if row['Airport C'] != 'Lego City Airport':
  #    return 'All other airports'
#las_all_chart['Airport C']=las_all_chart.apply(lambda row: label6(row), axis=1)
las_all_chart['Airport C']='All other airports' #function is not neccesary.

#table style
las_all_chart_style = las_all_chart.style.background_gradient() #style
dfi.export(las_all_chart_style,"Q4_table3.png") #export
las_all_chart_style #explore

In [ ]:
#merge the df's
airplane_ml =df_LAX_all.reset_index(drop=True).merge(df_LAS.reset_index(drop=True), left_index=True, right_index=True)
airplane_ml = airplane_ml[['Origin_x','Dest_x','DepDelay_x','ArrDelay_x','Month_x','DayofMonth_x','dep_hour_int_x','Origin_y','Dest_y','DepDelay_y','ArrDelay_y','Month_y','DayofMonth_y','dep_hour_int_y']]
airplane_ml['Month_x'] = airplane_ml['Month_x'].map({'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12})
airplane_ml['Month_y'] = airplane_ml['Month_y'].map({'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12})
airplane_ml

In [ ]:
#correlation heatmap
airplane_ml_heatm = sns.heatmap(airplane_ml.corr()) #plot the heatmap
airplane_ml_heatm
airplane_ml_heatm.figure.savefig("Q4_plot2.png")

In [ ]:
airplane_ml = pd.get_dummies(airplane_ml, columns=['Origin_x', 'Dest_x','Origin_y', 'Dest_y'])

#split into training and test set(80/20)
from sklearn.model_selection import train_test_split #import train_test_split function
X_train_cas, X_test_cas, y_train_cas, y_test_cas = train_test_split(airplane_ml.drop('ArrDelay_y', axis=1), airplane_ml['ArrDelay_y'], test_size=0.2, random_state=42)

In [ ]:
#import librarys and classifier
from sklearn.metrics import precision_score, roc_auc_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

#create model
model_cas = RandomForestClassifier(random_state=42)#42: ultimate question of life, the universe, and everything
model_cas.fit(X_train_cas, y_train_cas)

#probability and predictions
prob_cas = model_cas.predict_proba(X_test_cas)
train_pred_cas = model_cas.predict(X_train_cas)
pred_cas = model_cas.predict(X_test_cas)

#print scores:
print('Prediction: {:.5f}'.format(model_cas.score(X_test_cas, y_test_cas)))
print('Accuracy: {:.5f}'.format(accuracy_score(y_test_cas, pred_cas)))
print('Precision: {:.5f}'.format(precision_score(y_train_cas, train_pred_cas)))
print('Recall: {:.5f}'.format(recall_score(y_test_cas, pred_cas)))


In [ ]:
#plot 
from sklearn import metrics

y_pred_proba = model_cas.predict_proba(X_test_cas)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test_cas,  y_pred_proba)
auc = metrics.roc_auc_score(y_test_cas, y_pred_proba)

f, ax = plt.subplots(figsize=(10, 7))
plt.plot(fpr,tpr,label="AUC score:"+str(auc), alpha=0.8,linewidth=3)
plt.legend(loc=4, fontsize =15)
ax.set_title('Model that predicts cascading delay failures', fontsize=20, fontname="Monospace", alpha=.8)
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout#tight layout
plt.savefig('Q4_plot1.png', bbox_inches='tight')
plt.show() #plot

In [ ]:
ml1 = pd.read_csv("2006.csv")
ml2 = pd.read_csv("2007.csv")
#ml3 = pd.read_csv("2008.csv") #ignore data due to missing months

#if a flight is cancelled, then delay entry shouldn't be possible
ml1=ml1[ml1['Cancelled']==0] #select only not cancelled flights for 2007.
ml2=ml2[ml2['Cancelled']==0] #select only not cancelled flights for 2008.
ml1=ml1[ml1['Diverted']==0] #select only not diverted flights for 2007.
ml2=ml2[ml2['Diverted']==0] #select only not diverted flights for 2008.

print(ml1.shape)
print(ml2.shape)

In [ ]:
#take sample to match datapoints (avoid bias)
ml2 = ml2.sample(7019988)
print(ml2.shape)
#combine data
ml = pd.concat([ml1, ml2],ignore_index=True) #combine dataframes
#create a total delay column
ml['delay_total']= (ml['ArrDelay']+ml['DepDelay'])
#change timeformat of the 'CRSDepTime' column
ml['CRSDepTime']=(ml['CRSDepTime']/100)
ml['CRSArrTime']=(ml['CRSArrTime']/100)
#create delay column with multiclass values.
delay =[]
for row in ml['delay_total']:
    if row > 45:
        delay.append(2)    #more than 45 min late (total delay)
    elif row < 45:  
        delay.append(1)    #less than 45 min late (total delay)
    else:
        delay.append(0)    #ontime or arriving before time (total delay)
ml['delay'] = delay 

ml #explore data

In [ ]:
#no. of total delays above 45 min(2), delays below 45 min (1), and on time or arriving before (0)
ml.value_counts('delay')

In [ ]:
#slect relevant columns & check for NA's
ml = ml[['Month','DayOfWeek','DayofMonth','Origin','Dest','CRSDepTime','DepDelay','CRSArrTime','ArrDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','delay']]
ml.isnull().sum()

In [ ]:
#correlation heatmap
import seaborn as sns #imports the seaborn library
sns.heatmap(ml.corr()) #plot the heatmap

In [ ]:
#take a sample of the data (due to computational limitations)
ml=ml.sample(250000)

#dummies from orgigin and dest columns
ml = pd.get_dummies(ml, columns=['Origin', 'Dest'])
ml.head()

In [ ]:
#split into training and test set(80/20)
from sklearn.model_selection import train_test_split #import  function
X_train, X_test, y_train, y_test = train_test_split(ml.drop('delay', axis=1), ml['delay'], test_size=0.2, random_state=42)

### Model 1: Gradient Boosting Classifier

In [ ]:
#import librarys and classifier
from sklearn.metrics import precision_score, roc_auc_score, recall_score, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

#create model
model1 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)

#probability and predictions
prob1 = model1.predict_proba(X_test)
train_pred1 = model1.predict(X_train)
pred1 = model1.predict(X_test)

#print scores:
print('Prediction: {:.5f}'.format(model1.score(X_test, y_test)))
print('Accuracy: {:.5f}'.format(accuracy_score(y_test, pred1)))
print('AUC score: {:.5f}'.format(roc_auc_score(y_test, prob1, multi_class='ovr')))
print('Precision: {:.5f}'.format(precision_score(y_train, train_pred1, average='micro')))
print('Recall: {:.5f}'.format(recall_score(y_test, pred1, average='micro')))

### Model 2: Random Forest Classifier

In [ ]:
#import classifier
from sklearn.ensemble import RandomForestClassifier

#create model
model2 = RandomForestClassifier(random_state=42)#42: ultimate question of life, the universe, and everything
model2.fit(X_train, y_train)

#probability and predictions
prob2 = model2.predict_proba(X_test)
train_pred2 = model2.predict(X_train)
pred2 = model2.predict(X_test)

#print scores:
print('Prediction: {:.5f}'.format(model2.score(X_test, y_test)))
print('Accuracy: {:.5f}'.format(accuracy_score(y_test, pred2)))
print('AUC score: {:.5f}'.format(roc_auc_score(y_test, prob2, multi_class='ovr')))
print('Precision: {:.5f}'.format(precision_score(y_train, train_pred2, average='micro')))
print('Recall: {:.5f}'.format(recall_score(y_test, pred2, average='micro')))

### Model 3: Decision Tree Classifier

In [ ]:
#import classifier
from sklearn.tree import DecisionTreeClassifier

#create model
model3 = DecisionTreeClassifier(random_state=42)
model3.fit(X_train, y_train)

#probability and predictions
prob3 = model3.predict_proba(X_test)
train_pred3 = model3.predict(X_train)
pred3 = model3.predict(X_test)

#print scores:
print('Prediction: {:.5f}'.format(model3.score(X_test, y_test)))
print('Accuracy: {:.5f}'.format(accuracy_score(y_test, pred3)))
print('AUC score: {:.5f}'.format(roc_auc_score(y_test, prob3, multi_class='ovr')))
print('Precision: {:.5f}'.format(precision_score(y_train, train_pred3, average='micro')))
print('Recall: {:.5f}'.format(recall_score(y_test, pred3, average='micro')))

###  Random Forest.

In [ ]:
df_bin = pd.concat([df1, df2],ignore_index=True) #combine dataframes

In [ ]:
df_bin['delay_total']= (df_bin['ArrDelay']+df_bin['DepDelay'])
df_bin['CRSDepTime']=(df_bin['CRSDepTime']/100)
df_bin['CRSArrTime']=(df_bin['CRSArrTime']/100)
#binary delay column.
delay_bin =[]
for row in df_bin['delay_total']:
    if row > 0:
        delay_bin.append(1)    #delayed (total delay)
    else:
        delay_bin.append(0)    #on time or arriving before time (total delay)
df_bin['delay'] = delay_bin

In [ ]:
df_bin = df_bin[['Month','DayOfWeek','DayofMonth','Origin','Dest','CRSDepTime',
                 'DepDelay','CRSArrTime','ArrDelay','CarrierDelay','WeatherDelay',
                 'NASDelay','SecurityDelay','LateAircraftDelay','delay']] #select cols
df_bin = df_bin.replace(np.nan, 0) #replace nan with 0 (hence no flight)
df_bin=df_bin.sample(250000) #sample
df_bin_ml = pd.get_dummies(df_bin, columns=['Origin', 'Dest']) #dummy 
df_bin_ml.head() #explore

In [ ]:
#test/train split
from sklearn.model_selection import train_test_split #import train_test_split function
X_train_BIN, X_test_BIN, y_train_BIN, y_test_BIN = train_test_split(df_bin_ml.drop('delay', axis=1), df_bin_ml['delay'], test_size=0.2, random_state=42)


In [ ]:
#import librarys and classifier
from sklearn.metrics import precision_score, roc_auc_score, recall_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

#create model
model4 = RandomForestClassifier(random_state=42)#42: ultimate question of life, the universe, and everything
model4.fit(X_train_BIN, y_train_BIN)

#probability and predictions
prob_BIN = model4.predict_proba(X_test_BIN)
train_pred_BIN = model4.predict(X_train_BIN)
pred_BIN = model4.predict(X_test_BIN)

#print scores:
print('Prediction: {:.5f}'.format(model4.score(X_test_BIN, y_test_BIN)))
print('Accuracy: {:.5f}'.format(accuracy_score(y_test_BIN, pred_BIN)))
print('Precision: {:.5f}'.format(precision_score(y_train_BIN, train_pred_BIN)))
print('Recall: {:.5f}'.format(recall_score(y_test_BIN, pred_BIN)))


In [ ]:
# Set the colour palette
sns.reset_orig()
my_palette = sns.color_palette("colorblind")
plt.style.use('seaborn-colorblind')
sns.set_style("whitegrid")


#plot 
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

y_pred_proba = model4.predict_proba(X_test_BIN)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test_BIN,  y_pred_proba)
auc = metrics.roc_auc_score(y_test_BIN, y_pred_proba)

f, ax = plt.subplots(figsize=(10, 7))
plt.plot(fpr,tpr,label="AUC score:"+str(auc))
plt.legend(loc=4,fontsize = 15)
ax.set_title('Binary Random Forrest model',fontsize=20, fontname="Monospace", alpha=.8)
plt.tight_layout(rect=[0, 0, 1, 1]) #tight layout
plt.savefig('Q5_plot2.png', bbox_inches='tight')
plt.show() #plot
